In [1]:
# setup
from pyswmm import Simulation
import os
import dask

In [2]:
# nsims
nsims = 5

# specify location
print(os.path.abspath(os.curdir))
os.chdir("..")
dir_path = os.path.abspath(os.curdir)
print(dir_path)

input_path = dir_path + r'\input\swmm\input_'
print(input_path)

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\JS_temp
C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides
C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\input\swmm\input_


## Step 1: make this into a decorated function that can be thrown into a for loop

In [ ]:
'''
# run swmm, for each sim
for i in range(1, nsims+1):
    sim_folder = input_path + str(i)
    sim_file = os.path.join(sim_folder, "NPlesantCreek.inp")

    binary_path = sim_folder + r'\NPlesantCreek.out'

    # delete pre-existing .out in order to run swmm
    if os.path.exists(binary_path):
        os.remove(binary_path)
    else:
        print("Can not delete the file as it doesn't exists")

    # load the model - no interaction, write out binary file
    sim = Simulation(inputfile=sim_file, reportfile=None, outputfile=binary_path)
    sim.execute()'''

In [4]:
@dask.delayed
def do_job(i):
    sim_folder = input_path + str(i)
    sim_file = os.path.join(sim_folder, "NPlesantCreek.inp")

    binary_path = sim_folder + r'\NPlesantCreek.out'

    # delete pre-existing .out in order to run swmm
    if os.path.exists(binary_path):
        os.remove(binary_path)
    else:
        print("Can not delete the file as it doesn't exist")

    # load the model - no interaction, write out binary file
    sim = Simulation(inputfile=sim_file, reportfile=None, outputfile=binary_path)
    sim.execute()

In [5]:
delayed_jobs = [do_job(a) for a in range(1,nsims+1)]

In [8]:
%%time
dask.delayed(lambda x: x)(delayed_jobs).compute()

Can not delete the file as it doesn't existsCan not delete the file as it doesn't exists
Can not delete the file as it doesn't exists

Can not delete the file as it doesn't exists
Can not delete the file as it doesn't exists


SWMMException: 
  ERROR 303: cannot open input file.